In [1]:
%matplotlib widget
from pyLinViscoFit import inter
GUI = inter.Control()

# Prony series identification for linear viscoelastic material models
Martin Springer | 2022 v0.0.1 

***
## Overview

Linear viscoelastic materials are often described with a Generalized Maxwell model. The necessary model parameter are identified by fitting a Prony series to the experimental measurement data. 

This Jupyter notebook allows for the identification of Prony series parameter from experimental data measured in either the frequency domain (via Dynamic Mechanical Thermal Analysis) or time domain (via relaxation measurements). The experimental data can be provided as raw measurement sets at different temperatures or as pre-processed master curves.

* If raw measurement data are provided, the time-temperature superposition principle is applied to create a master curve and obtain shift functions before the Prony series parameter are identified. 

* If master curves are provided, the shift procedure can be skipped and the Prony series parameters can be identified directly. 

An optional minimization routine is provided at the end of this notebook to reduce the number of Prony elements. This routine is helpful for Finite Element simulations where reducing the computational complexity of the linear viscoelastic material models can reduce the simulation time. 

In [2]:
display(GUI.b_theory)
display(GUI.out_theory)

ToggleButton(value=False, description='Click here for more details!', layout=Layout(width='200px'))

HTMLMath(value='')

***
## Parameter identification


### Specify measurement type and upload input data

In this section the measurement type is specified and the experimental data are uploaded. A set of example input files can be downloaded here: [Example input files](https://github.com/martin-springer/LinViscoFit/raw/main/examples/examples.zip)

#### Conventions 
| Physical quantity        | Symbol               | Variable   | Unit   |
| :---------------------   | :-------------:      | :--------- | :----: |
| Relaxation modulus:      | $E(t)$               | `E_relax`  | MPa    |
| Storage modulus:         | $E'(\omega)$         | `E_stor`   | MPa    |
| Loss modulus:            | $E''(\omega)$        | `E_loss`   | MPa    |
| Complex modulus:         | $\lvert E^{*}\rvert$ | `E_comp`   | MPa    |
| Loss modulus:            | $\tan(\delta)$       | `tan_del`  | -      |
| Instantaneous modulus:   | $E_0$                | `E_0`      | MPa    |
| Equilibrium modulus:     | $E_{inf}$            | `E_inf`    | MPa    |
| Angular frequency:       | $\omega$             | `omega`    | rad/s  |
| Frequency:               | $f$                  | `f`        | 1/s    |
| Time:                    | $t$                  | `t`        | s      |
| Temperature:             | $\theta$             | `T`        | °C     |
| Relaxation times:        | $\tau  _i$           | `tau_i`    | s      |
| Relaxation moduli:       | $E_i$                | `E_i`      | MPa    |
| Norm. relaxation moduli: | $\alpha_i$           | `alpha_i`  | -      |

#### Domain
This notebook allows for the estimation of Prony series parameters used in a Generalized Maxwell model to describe  linear viscoelastic material behavior. The parameters can be either fitted from measurement data of Dynamic mechanical thermal analysis (DMTA) in the frequency domain (freq) or from relaxation experiments in the time domain (time). 

#### Instrument
DMTA measurements conducted with a Netzsch Gabo DMA EPLEXOR (Eplexor) can be directly uploaded as Excel files. Use the `Excel Export!` feature of the Eplexor software with the default template to create the input files. For measurements conducted with other instruments choose the user option (user) and prepare the input as comma-separated values (csv) files with the following column headers:
* **Frequency domain:** header = `f, E_stor, E_loss`, where `f` is the frequency in Hertz (Hz), and ```E_stor``` and `E_loss` are the storage and loss modulus in Megapascal (MPa), respectively.
* **Time domain:** header = `t, E_relax`, where `t` is the time in seconds (s), and `E_relax` is the relaxation modulus in MPa.

#### Type
The data can be provided as measurement sets at different temperatures (raw) or as master curve obtained from time-temperature superposition (master). For raw measurement data the individual temperature sets need to be identified:
* **Eplexor:** The notebook identifies the corresponding temperature sets automatically (only available in the frequency domain).
* **user:** Two additional columns need to be included in the input file. One column describing the temperature `T` of the measurement point and a second column `Set` to identify the corresponding measurement set of the data point. All measurement points at the same temperature level are marked with the same number, e.g. 0 for the first measurement set. The first measurement set (0) represents the coldest temperature followed by the second set (1) at the next higher temperature level and so forth (see the provided [example input file](https://github.com/martin-springer/LinViscoFit/blob/main/examples/time_user_raw.csv) for further details).

In [3]:
display(GUI.w_inp_gen)

#### Reference temperature 

The temperature for which the master curve has been created through time-temperature superpostion. 

* **Eplexor:** The reference temperature will be automatically extracted from the input file. 
* **user:** The reference temperature needs to be specified below.
  - **master:** Reference temperature of the provided master curve. 
  - **raw:** Desired reference temperature for the master curve. (The provided temperature will be automatically adjusted to  the closest available temperature level of the measurement sets.)

#### Optional shift factor upload
* **master:** Uploading the shift factors allows for the calculation of polynomial (D1 to D4) shift functions and the Williams–Landel–Ferry (WLF) shift function, but is not required for the Prony series estimation. 
> _**Note**_: If a master curve from the Eplexor software is provided, the default behavior of the notebook is to use the WLF shift function from the Eplexor software. However, in the time-temperature superpostion section, a checkbox is provided to overwrite the WLF fit of the Eplexor software and conduct another WLF fit with the algorithm in this notebook.

* **raw:** The shift factors can be either directly determined for the desired reference temperature in the time-temperature superposition section (no upload necessary) or user-specified shift factors can be uploaded to be used to create the master curve. 

In [4]:
display(GUI.w_inp_shift)

#### Load and check the provided input files

Once all the input parameter are specified and the necessary files are uploaded, click the `Load data` button below. The widgets indicate the detected data.

In [5]:
display(GUI.w_inp_load)

***
### Time-temperature superposition (shift functions)

This section allows the calculation of shift factors from raw input data to create a master curve. The shift factors are used to fit polynomial and WLF shift functions.

#### Shift factors $\log(a_{T})$ - Create master curve from raw input



The time-temperature superposition principle is applied to create a master curve from the individual  measurement sets at different temperature levels. 

> _**Note**_:This subsection only applies to raw measurement data. If a master curve was uploaded, procede to the next step. 

  * **user shift factors uploaded:** The provided shift factors will be used to create the master curve from the raw measurement sets (_**Note**_: the checkbox below allows to overwrite the uploaded user shift factors and fit new ones). 
  * **No user shift factors uploaded:** The measurement sets from the raw input file are used to estimate the shift factors and create a master curve. Measurement sets below the desired reference temperatures are shifted to lower frequencies (longer time periods), whereas measurement sets at temperatures higher than the reference temperature are shifted to higher frequencies (shorter time periods). (_**Note**_: In the frequency domain, only the storage modulus input data are considered to create the master curve from the raw input data. The shift factors obtained from the storage modulus master curve are then used to create the loss modulus master curve.)

In [6]:
display(GUI.w_aT)

#### Shift functions (WLF & Polynomial degree 1 to 4)

If shift factors are available, the WLF shift function and polynomial functions of degree 1 to 4 can be fitted and plotted below. (_**Note**_: If the WLF shift functions was already provided by the Eplexor software, the checkbox below let's you overwrite the WLF fit of the Eplexor software with a WLF fit of this notebook.) 

> _**Note**_:This subsection only provides shift functions and is not required to perform the parameter identification of the Prony series.

In [7]:
display(GUI.w_shift)

***
### Estimate Prony series parameters

#### Pre-process (smooth) master curve

A moving median filter to remove outliers in the measurement data can be applied before the Prony series parameters are identified. The window size can be adjusted through the slider above the figure. A window size of 1 means that no filtering procedure is performed and the raw data are fitted.

In [8]:
display(GUI.b_smooth)
display(GUI.out_smooth)

Button(button_style='info', description='smooth master curve', layout=Layout(height='auto', width='200px'), st…

Output()

#### Define the number and discretization of the Prony series

The number of Prony terms, $N$, needs to be defined before the parameter $\tau_i$ and $\alpha_i$ can be identified. The `default` behavior is to equally space one Prony term per decade along the logarithmic time axis, $\tau_i$ = [1E-1, 1E0, 1E1,...] (s). This discretization typically delivers accurate results for engineering applications. 
> _**Note:**_ The fine discretization can be computationally heavy for using the viscoelastic material models in Finite Element simulations. Hence, the default discretization can be modified by either using the optimization routine provided below or by manually defining the number of Prony terms (`manual`). Additionally, the user can decide whether to round the lowest and highest relaxation times, $\tau_i$, to the nearest base 10 number within the measurement window `round` or to use the exact minimum and maximum values of the experimental data for the relaxation times `exact`. 

In [9]:
display(GUI.w_dis)

#### Curve fitting

Two different curve fitting routines for the Prony series parameters are employed and are dependent on the domain of the input data:

* **Frequency domain**: A generalized collocation method using stiffness matrices is used as described in [Kraus, M. A., and M. Niederwald. Eur J Eng Mech 37.1 (2017): 82-106](https://journals.ub.ovgu.de/index.php/techmech/article/view/600). This methods utilizes both the storage and loss modulus master curves to estimate the Prony series parameters.
   
   
* **Time domain**: A least-squares minimization is performed with the L-BFGS-B method from the scipy package. The implementation is similar to the optimization problem described by [Barrientos, E., Pelayo, F., Noriega, Á. et al. Mech Time-Depend Mater 23, 193–206 (2019)](https://doi.org/10.1007/s11043-018-9394-z) for a homogenous distribution of discrete times. 

In [10]:
display(GUI.b_fit)
display(GUI.w_out_fit_prony)

Button(button_style='info', description='fit Prony series', layout=Layout(height='auto', width='200px'), style…

In [11]:
tau = GUI.prony['df_terms']['tau'].values
alpha = GUI.prony['df_terms']['alpha'].values
time = GUI.df_master['t'].values

AttributeError: 'Control' object has no attribute 'prony'

In [ ]:
%%timeit -r1 -n1
for i in range(10000):
#Reference
    visco.prony.E_relax_norm(time, alpha, tau)

In [ ]:
%load_ext Cython

In [ ]:
%%cython -3 -n cE_relax --cplus --compile-args=/openmp 
#--annotate

cimport cython
cimport numpy as np
import numpy as np

from cython.parallel import prange
from libc.math cimport exp 

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
def cE_relax_norm(np.ndarray[double, ndim=1] time, 
                  np.ndarray[double, ndim=1] alpha_i,
                  np.ndarray[double, ndim=1] tau_i):
   
    cdef int i, j
    cdef np.ndarray [np.float64_t, ndim=1] out = np.empty_like(time, dtype=float)
    cdef double temp_sum  

    for i in prange(time.shape[0], nogil=True):
        temp_sum = 0
        for j in range(alpha_i.shape[0]):
            temp_sum = temp_sum + alpha_i[j]*(1-exp(-time[i]/tau_i[j]))
        out[i] = 1 - temp_sum
    return out

In [ ]:
%%timeit -r1 -n1
for i in range(10000):
    cE_relax_norm(time, alpha, tau)

In [ ]:
from pyLinViscoFit import cython

In [ ]:
%%timeit -r1 -n1
for i in range(10000):
    cython.cE_relax_norm.func(time, alpha, tau)

***
#### Generalized Maxwell model

The fitted Prony series parameters in combination with the Generalized Maxwell model can be used to calculate the linear viscoelastic material parameters in both the time and frequency domain. 

In [12]:
display(GUI.b_GMaxw)
display(GUI.out_GMaxw)

Button(button_style='info', description='plot Generalized Maxwell', layout=Layout(height='auto', width='200px'…

Output()

***
### Optional: Minimize number of Prony elements (for Finite Element simulations)

The Generalized Maxwell model in combination with a high number of Prony terms can be computationally expensive. Especially, when used in combination with numerical frameworks as the Finite Element Method. Reducing the number of Prony elements will decrease the accurace of the linear viscoelastic material model, but can help to speed up subsequent numerical simulations.

We provide a simple routine to create an additional Generalized Maxwell model with a reduced number of Prony elements. The routine starts with the number of Prony terms specified above and subsequently reduces the number of terms. A least squares minimization is performed to fit the reduced term Prony parameters. The least squares residual is used to suggest an optimal number of Prony terms for subsequent FEM simulations ($R^2 \approx$ 0.025). However, the user can change this default setting by selecting a different number of Prony terms below.

> **_Note:_** This routine is computationally more demanding and can take a few minutes to complete. The runtime depends on the initial number of Prony elements and the number of data points in the measurement sets.

In [13]:
display(GUI.b_opt)
display(GUI.w_out_fit_min)

Button(button_style='warning', description='minimize Prony terms', layout=Layout(height='auto', width='200px')…

In [14]:
%%timeit
inter.opt.nprony(GUI.df_master, GUI.prony, window='min')

Prony series fit N = 30: Succesful!
Prony series fit N = 28: Succesful!
Prony series fit N = 26: Succesful!
Prony series fit N = 24: Succesful!
Prony series fit N = 22: Succesful!
Prony series fit N = 20: Succesful!
Prony series fit N = 19: Succesful!
Prony series fit N = 18: Succesful!
Prony series fit N = 17: Succesful!
Prony series fit N = 16: Succesful!
Prony series fit N = 15: Succesful!
Prony series fit N = 14: Succesful!
Prony series fit N = 13: Succesful!
Prony series fit N = 12: Succesful!
Prony series fit N = 11: Succesful!
Prony series fit N = 10: Succesful!
Prony series fit N = 09: Succesful!
Prony series fit N = 08: Succesful!
Prony series fit N = 07: Succesful!
Prony series fit N = 06: Succesful!
Prony series fit N = 05: Succesful!
Prony series fit N = 04: Succesful!
Prony series fit N = 03: Succesful!
Prony series fit N = 30: Succesful!
Prony series fit N = 28: Succesful!
Prony series fit N = 26: Succesful!
Prony series fit N = 24: Succesful!


KeyboardInterrupt: 

***
### Download results

A zip archive including the identified Prony series parameters, (shift factors and shift functions), results of the Generalized Maxwell model, and figures can be dowloaded below.

In [ ]:
display(GUI.db_zip)
display(GUI.out_html)

***
### Start over! - Clear all input data and reload an empty notebook.

In [ ]:
display(GUI.b_reload)